In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf
%matplotlib inline

In [14]:
# read data
top_dir = '/Users/stiso/Documents/Code/graph_learning/ECoG_data/behavioral_data_raw/'
save_dir = '/Users/stiso/Documents/Code/graph_learning/ECoG_data/behavior_preprocessed/'
subj = [1,2,3,4,6,8,10,12,16]

In [15]:
#%% Load and concatenate data

data = pd.DataFrame()
for i in subj:
    curr = pd.read_csv("".join([top_dir, 'subj', str(i), '/subj', str(i), '_clean_data']))
    curr['subj'] = i
    data = data.append(curr)
    

/Users/stiso/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [16]:
data['subj'].unique()

array([ 1,  2,  3,  4,  6,  8, 10, 12, 16])

In [17]:
data.head(20)

,ISI_raw,Unnamed: 0,block,correct_raw,graph,hand,hand_transition,lag10,onset_raw,order,...,recency,resp,resp_raw,rt_raw,sess,subj,transition,trialNum,typing_raw,walk
0,0.05,5,1.0,1.0,lattice,left,False,1,7.087798,6.0,...,2.0,q,['q'],0.945559,1,1,False,6.0,a,2.0
1,0.05,8,1.0,1.0,lattice,right,False,1,10.337893,9.0,...,2.0,b,['b'],0.752210,1,1,False,9.0,a,0.0
2,0.05,9,1.0,1.0,lattice,right,False,1,11.187585,10.0,...,2.0,u,['u'],0.581672,1,1,False,10.0,a,8.0
3,0.05,12,1.0,1.0,lattice,right,True,0,13.455660,13.0,...,11.0,p,['p'],0.810152,1,1,False,13.0,a,6.0
4,0.05,13,1.0,1.0,lattice,right,False,2,14.357635,14.0,...,4.0,u,['u'],1.003771,1,1,False,14.0,a,8.0
5,0.05,14,1.0,1.0,lattice,right,False,2,15.457186,15.0,...,6.0,b,['b'],0.735995,1,1,False,15.0,a,0.0
6,0.05,15,1.0,1.0,lattice,right,False,3,16.288177,16.0,...,2.0,u,['u'],0.808980,1,1,False,16.0,a,8.0
7,0.05,16,1.0,1.0,lattice,left,True,1,17.190220,17.0,...,5.0,e,['e'],1.387098,1,1,False,17.0,a,7.0
8,0.05,17,1.0,1.0,lattice,left,False,1,18.674009,18.0,...,7.0,r,['r'],0.799790,1,1,False,18.0,a,9.0
9,0.05,18,1.0,1.0,lattice,left,False,0,19.574129,19.0,...,14.0,v,['v'],1.122980,1,1,False,19.0,a,1.0


In [18]:
# add session for subjects that dont have it
data.loc[data['sess'].isna(),'sess'] = 1

In [19]:
data.describe()

,ISI_raw,Unnamed: 0,block,correct_raw,lag10,onset_raw,order,pID,recency,rt_raw,sess,subj,trialNum,walk
count,10034.00,10034.000000,10034.000000,10034.000000,10034.000000,10034.000000,10034.000000,10034.000000,10034.000000,10034.000000,10034.000000,10034.000000,10034.000000,10034.000000
mean,0.05,489.803568,2.461232,0.982161,1.314232,543.641977,490.803568,6.656468,9.535380,0.823650,1.119294,7.031991,490.803568,4.412996
std,0.00,285.496104,1.116168,0.132374,1.068296,362.981873,285.496104,4.369449,11.972109,0.423954,0.324151,4.296273,285.496104,2.879772
min,0.05,1.000000,1.000000,0.000000,0.000000,2.003784,2.000000,1.000000,2.000000,0.172753,1.000000,1.000000,2.000000,0.000000
25%,0.05,239.000000,1.000000,1.000000,0.000000,232.940563,240.000000,3.000000,2.000000,0.583040,1.000000,3.000000,240.000000,2.000000
50%,0.05,487.000000,2.000000,1.000000,1.000000,510.301773,488.000000,6.000000,5.000000,0.733314,1.000000,6.000000,488.000000,4.000000
75%,0.05,736.000000,3.000000,1.000000,2.000000,783.662842,737.000000,12.000000,11.000000,0.949011,1.000000,12.000000,737.000000,7.000000
max,0.05,999.000000,4.000000,1.000000,5.000000,1587.839111,1000.000000,16.000000,168.000000,11.531556,2.000000,16.000000,1000.000000,9.000000


In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10034 entries, 0 to 546
Data columns (total 22 columns):
ISI_raw            10034 non-null float64
Unnamed: 0         10034 non-null int64
block              10034 non-null float64
correct_raw        10034 non-null float64
graph              10034 non-null object
hand               10034 non-null object
hand_transition    10034 non-null bool
lag10              10034 non-null int64
onset_raw          10034 non-null float64
order              10034 non-null float64
pID                10034 non-null float64
path               10034 non-null object
recency            10034 non-null float64
resp               10034 non-null object
resp_raw           10034 non-null object
rt_raw             10034 non-null float64
sess               10034 non-null int64
subj               10034 non-null int64
transition         10034 non-null bool
trialNum           10034 non-null float64
typing_raw         10034 non-null object
walk               10034 non-nu

In [21]:
data.to_csv("".join([save_dir, 'group_behavior.csv']))
